## Cleaning the Data

To start, I need to have a look at the data itself. I know from preliminary analysis done by the provider of the data at UC Irvine's ML Repository that some of the columns are not helpful for regression, so I need to ensure that my dataset is all numeric, and that any columns that contain non-regression related data are removed. 

Since this data is related to violent crimes, it is important to remember that correlation does not imply causation. This analysis does not aim to determine the cause of crime in any specific community or determine the cause for any specific type of crime.

In [84]:
import numpy as np
from scipy.linalg import svd
import seaborn as sns
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import GridSearchCV

In [85]:
# I will want to look at the data itself, so I add the column names to the datafram
column_names = ['state', 'county', 'community', 'communityname', 'fold', 'population', 'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29', 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam', 'PctLargHouseOccup', 'PersPerOccupHous', 'PersPerOwnOccHous', 'PersPerRentOccHous', 'PctPersOwnOccup', 'PctPersDenseHous', 'PctHousLess3BR', 'MedNumBR', 'HousVacant', 'PctHousOccup', 'PctHousOwnOcc', 'PctVacantBoarded', 'PctVacMore6Mos', 'MedYrHousBuilt', 'PctHousNoPhone', 'PctWOFullPlumb', 'OwnOccLowQuart', 'OwnOccMedVal', 'OwnOccHiQuart', 'RentLowQ', 'RentMedian', 'RentHighQ', 'MedRent', 'MedRentPctHousInc', 'MedOwnCostPctInc', 'MedOwnCostPctIncNoMtg', 'NumInShelters', 'NumStreet', 'PctForeignBorn', 'PctBornSameState', 'PctSameHouse85', 'PctSameCity85', 'PctSameState85', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps', 'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicAveOTWorked', 'LandArea', 'PopDens', 'PctUsePubTrans', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn', 'PolicBudgPerPop', 'ViolentCrimesPerPop']
data_raw = pd.read_csv('data/communities.data', sep=',')
data_raw.columns = column_names

In [86]:
# The original dataset included a data description that notes which columns are not predictive, so I will drop them here
dropped = ['state', 'county', 'community', 'communityname', 'fold']
data1 = data_raw.drop(axis = 1, columns = dropped)
data1.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 123 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    population             1993 non-null   float64
 1    householdsize          1993 non-null   float64
 2    racepctblack           1993 non-null   float64
 3    racePctWhite           1993 non-null   float64
 4    racePctAsian           1993 non-null   float64
 5    racePctHisp            1993 non-null   float64
 6    agePct12t21            1993 non-null   float64
 7    agePct12t29            1993 non-null   float64
 8    agePct16t24            1993 non-null   float64
 9    agePct65up             1993 non-null   float64
 10   numbUrban              1993 non-null   float64
 11   pctUrban               1993 non-null   float64
 12   medIncome              1993 non-null   float64
 13   pctWWage               1993 non-null   float64
 14   pctWFarmSelf           1993 non-null  

In [87]:
# Since some of the columns are listed as non-numeric, I will look at the data manually to see what type of data is inside
data1.head(10)

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.02,0.12,0.45,?,?,?,?,0.0,?,0.67
1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.01,0.21,0.02,?,?,?,?,0.0,?,0.43
2,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.02,0.39,0.28,?,?,?,?,0.0,?,0.12
3,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.04,0.09,0.02,?,?,?,?,0.0,?,0.03
4,0.02,0.28,0.06,0.54,1.00,0.25,0.31,0.48,0.27,0.37,...,0.01,0.58,0.10,?,?,?,?,0.0,?,0.14
5,0.01,0.39,0.00,0.98,0.06,0.02,0.30,0.37,0.23,0.60,...,0.05,0.08,0.06,?,?,?,?,0.0,?,0.03
6,0.01,0.74,0.03,0.46,0.20,1.00,0.52,0.55,0.36,0.35,...,0.01,0.33,0.00,?,?,?,?,0.0,?,0.55
7,0.03,0.34,0.20,0.84,0.02,0.00,0.38,0.45,0.28,0.48,...,0.04,0.17,0.04,?,?,?,?,0.0,?,0.53
8,0.01,0.40,0.06,0.87,0.30,0.03,0.90,0.82,0.80,0.39,...,0.00,0.47,0.11,?,?,?,?,0.0,?,0.15
9,0.13,0.71,0.15,0.07,1.00,0.41,0.40,0.52,0.35,0.33,...,0.02,1.00,1.00,?,?,?,?,0.0,?,0.24


In [88]:
# Pandas doesn't recognize '?' as a null character, so I need to replace it with a pandas recognized null character.
data2 = data1.replace('?', np.nan) #Replace '?' with NaN values so I can use built-in pandas functions.
data2.head(5)

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.0,NaN,0.67
1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.0,NaN,0.43
2,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.02,0.39,0.28,NaN,NaN,NaN,NaN,0.0,NaN,0.12
3,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.04,0.09,0.02,NaN,NaN,NaN,NaN,0.0,NaN,0.03
4,0.02,0.28,0.06,0.54,1.00,0.25,0.31,0.48,0.27,0.37,...,0.01,0.58,0.10,NaN,NaN,NaN,NaN,0.0,NaN,0.14


In [89]:
# Now I can get a better sense of the missing data
data2.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 123 columns):
 #    Column                 Non-Null Count  Dtype  
---   ------                 --------------  -----  
 0    population             1993 non-null   float64
 1    householdsize          1993 non-null   float64
 2    racepctblack           1993 non-null   float64
 3    racePctWhite           1993 non-null   float64
 4    racePctAsian           1993 non-null   float64
 5    racePctHisp            1993 non-null   float64
 6    agePct12t21            1993 non-null   float64
 7    agePct12t29            1993 non-null   float64
 8    agePct16t24            1993 non-null   float64
 9    agePct65up             1993 non-null   float64
 10   numbUrban              1993 non-null   float64
 11   pctUrban               1993 non-null   float64
 12   medIncome              1993 non-null   float64
 13   pctWWage               1993 non-null   float64
 14   pctWFarmSelf           1993 non-null  

In [90]:
# Since the columns with missing data are missing more than 50% of possible values I drop them so I have a robust dataset
dropped = ['OtherPerCap', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps', 'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic', 'PolicPerPop', 'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp', 'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicAveOTWorked', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr', 'LemasGangUnitDeploy', 'PolicBudgPerPop']
data3 = data2.drop(axis = 1, columns = dropped)
data3.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 100 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   population             1993 non-null   float64
 1   householdsize          1993 non-null   float64
 2   racepctblack           1993 non-null   float64
 3   racePctWhite           1993 non-null   float64
 4   racePctAsian           1993 non-null   float64
 5   racePctHisp            1993 non-null   float64
 6   agePct12t21            1993 non-null   float64
 7   agePct12t29            1993 non-null   float64
 8   agePct16t24            1993 non-null   float64
 9   agePct65up             1993 non-null   float64
 10  numbUrban              1993 non-null   float64
 11  pctUrban               1993 non-null   float64
 12  medIncome              1993 non-null   float64
 13  pctWWage               1993 non-null   float64
 14  pctWFarmSelf           1993 non-null   float64
 15  pct

## Preparing the data for modeling
Now the data is numeric and setup in a pandas dataframe. The size of the dataset is relatively small, so analysis in a timely manner will not be an issue. Although the dataset is a large number of columns, I don't think that dimesnion reduction by a Principal Component Analysis is needed. As such, The data is almost ready for modeling. In this section, I split the data into target and feature sets and split it into training and test sets. 

In [91]:
# I need to separate the target value from the other columns and split the data into test and training sets
X = data3.drop(['ViolentCrimesPerPop'], axis = 1)
y = data3['ViolentCrimesPerPop']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

## Modeling the data
Here I will test threee models from the SKlearn library: Linear, Lasso, and Ridge regression. I will use the default values for the linear model to use it as a baseline, but every model will be standardized, run through a polnomial feature operation, and optimized with GridSearchCV. I will also use 5-fold cross validation and compare the performance. 

Models wil be scored with R2 scoring, where scores range from 1 to -inf, with 1 being a perfect score. Any model with a negative 'mean_test_score' is essentially useless.

### Linear Regression

In [92]:
# I want a baseline for my regression, so I fit a simple linear regression with standard settings.
simple_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures()), 
    ('regressor', LinearRegression())])
parameters = {'poly_features__degree': [1, 2, 3]}
simple_grid = GridSearchCV(simple_pipe, parameters).fit(X_train, y_train)

In [93]:
# Below I display the results of the grid search
simple_model_results = pd.DataFrame(simple_grid.cv_results_)
simple_model_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_poly_features__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.039963,0.021441,0.003405,0.000842,1,{'poly_features__degree': 1},0.663081,0.674546,0.675946,0.557779,0.700575,0.654386,0.049826,1
1,1.764026,0.055452,0.015320,0.001579,2,{'poly_features__degree': 2},-0.001102,-0.339856,-0.123749,-0.759552,-0.315404,-0.307932,0.258110,3
2,37.100861,0.592366,0.290955,0.052374,3,{'poly_features__degree': 3},0.056520,-0.043663,-0.558333,-0.757515,-0.129224,-0.286443,0.315315,2


In [94]:
# For later comparison, I pull out the best parameters that yielded the best model
simple_model_best_params = simple_grid.best_params_
simple_model_best_params

{'poly_features__degree': 1}

### Lasso Regression

In [95]:
#Performing a Lasso regression with optimized hyperparameters and analyzing the results.
lasso_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures()), 
    ('lasso', Lasso())])
parameters = {'poly_features__degree':[1, 2, 3], 'lasso__alpha': [0.01, 0.1, 1, 10]}
lasso_grid = GridSearchCV(lasso_pipe, param_grid = parameters).fit(X_train, y_train)

In [96]:
# Below I display the results of the grid search
lasso_model_results = pd.DataFrame(lasso_grid.cv_results_)
lasso_model_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lasso__alpha,param_poly_features__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014631,0.001361,0.004344,0.001084,0.01,1,"{'lasso__alpha': 0.01, 'poly_features__degree'...",0.644162,0.671351,7.099923e-01,0.555374,0.667988,0.649773,0.051704,2
1,0.680754,0.225013,0.018406,0.003701,0.01,2,"{'lasso__alpha': 0.01, 'poly_features__degree'...",0.650225,0.651188,7.281127e-01,0.558115,0.674489,0.652426,0.054984,1
2,144.758733,4.952722,0.303124,0.027786,0.01,3,"{'lasso__alpha': 0.01, 'poly_features__degree'...",0.614036,0.475354,5.407308e-01,0.489023,0.668369,0.557502,0.073765,3
3,0.016012,0.003251,0.003833,0.000771,0.1,1,"{'lasso__alpha': 0.1, 'poly_features__degree': 1}",0.398851,0.405168,3.735648e-01,0.366624,0.403609,0.389563,0.016182,6
4,0.200044,0.021203,0.014790,0.001867,0.1,2,"{'lasso__alpha': 0.1, 'poly_features__degree': 2}",0.377564,0.416975,4.832826e-01,0.344648,0.367603,0.398015,0.048623,5
5,23.075066,5.021584,0.275519,0.013716,0.1,3,"{'lasso__alpha': 0.1, 'poly_features__degree': 3}",0.478406,0.483257,5.826768e-01,0.401167,0.490413,0.487184,0.057655,4
6,0.011809,0.004246,0.004076,0.000849,1,1,"{'lasso__alpha': 1, 'poly_features__degree': 1}",-0.005523,-0.000809,-5.041186e-07,-0.008766,-0.000022,-0.003024,0.003529,8
7,0.130683,0.009614,0.018018,0.004565,1,2,"{'lasso__alpha': 1, 'poly_features__degree': 2}",-0.005523,-0.000809,-5.041186e-07,-0.008766,-0.000022,-0.003024,0.003529,8
8,4.965245,0.278763,0.269180,0.018567,1,3,"{'lasso__alpha': 1, 'poly_features__degree': 3}",0.022162,0.061328,1.469803e-01,0.045702,0.049346,0.065104,0.042864,7
9,0.015410,0.006246,0.004733,0.001552,10,1,"{'lasso__alpha': 10, 'poly_features__degree': 1}",-0.005523,-0.000809,-5.041186e-07,-0.008766,-0.000022,-0.003024,0.003529,8


In [97]:
# For later comparison, I pull out the best parameters that yielded the best model
lasso_model_best_params = lasso_grid.best_params_
lasso_model_best_params

{'lasso__alpha': 0.01, 'poly_features__degree': 2}

### Ridge Regression

In [98]:
#Performing a Lasso regression with optimized hyperparameters and analyzing the results.
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly_features', PolynomialFeatures()), 
    ('ridge', Ridge())])
parameters = {'poly_features__degree':[1, 2, 3], 'ridge__alpha': [0.01, 0.1, 1, 10]}
ridge_grid = GridSearchCV(ridge_pipe, param_grid = parameters).fit(X_train, y_train)

In [99]:
# Below I display the results of the grid search
ridge_model_results = pd.DataFrame(ridge_grid.cv_results_)
ridge_model_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_poly_features__degree,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.022769,0.005341,0.004314,0.000468,1,0.01,"{'poly_features__degree': 1, 'ridge__alpha': 0...",0.663262,0.674537,0.676072,0.557814,0.700572,0.654451,0.049828,4
1,0.017800,0.005717,0.006052,0.002762,1,0.1,"{'poly_features__degree': 1, 'ridge__alpha': 0.1}",0.664649,0.674464,0.677145,0.558086,0.700542,0.654977,0.049857,3
2,0.009697,0.000769,0.002655,0.000595,1,1,"{'poly_features__degree': 1, 'ridge__alpha': 1}",0.669679,0.674343,0.684459,0.559354,0.700583,0.657684,0.050294,2
3,0.009304,0.000860,0.002530,0.000672,1,10,"{'poly_features__degree': 1, 'ridge__alpha': 10}",0.670622,0.676596,0.702764,0.562122,0.702470,0.662915,0.052074,1
4,0.231549,0.012175,0.019715,0.007194,2,0.01,"{'poly_features__degree': 2, 'ridge__alpha': 0...",-0.000669,-0.340720,-0.128498,-0.752377,-0.315038,-0.307460,0.255054,12
5,0.258488,0.027391,0.018705,0.005410,2,0.1,"{'poly_features__degree': 2, 'ridge__alpha': 0.1}",0.003033,-0.335391,-0.124672,-0.747942,-0.310328,-0.303060,0.254768,11
6,0.251193,0.040821,0.025595,0.009628,2,1,"{'poly_features__degree': 2, 'ridge__alpha': 1}",0.037330,-0.286527,-0.089465,-0.706489,-0.267229,-0.262476,0.252018,6
7,0.231161,0.013232,0.017966,0.004149,2,10,"{'poly_features__degree': 2, 'ridge__alpha': 10}",0.235076,-0.017600,0.112531,-0.448424,-0.030058,-0.029695,0.230570,5
8,4.130350,0.060172,0.334550,0.045974,3,0.01,"{'poly_features__degree': 3, 'ridge__alpha': 0...",0.056070,-0.042627,-0.607099,-0.758758,-0.134275,-0.297338,0.324106,10
9,4.260075,0.110847,0.329650,0.025812,3,0.1,"{'poly_features__degree': 3, 'ridge__alpha': 0.1}",0.056095,-0.042609,-0.607065,-0.758722,-0.134255,-0.297311,0.324100,9


In [100]:
# For later comparison, I pull out the best parameters that yielded the best model
ridge_model_best_params = ridge_grid.best_params_
ridge_model_best_params

{'poly_features__degree': 1, 'ridge__alpha': 10}

## Comparing Model Performance
Looking at the three models, all three models performed comparably. Below is a table of the performance of each model along with teh 

In [115]:
#Creating a dataframe to display the results
pd.set_option('display.max_colwidth', None) # Stop the display from truncating values for best params
results_df = pd.DataFrame([
    ['Linear', simple_grid.best_score_, simple_model_best_params],
    ['Lasso', lasso_grid.best_score_, lasso_model_best_params],
    ['Ridge', ridge_grid.best_score_, ridge_model_best_params]
], columns = ['Model', 'Best Score', 'Best Params'])
results_df

,Model,Best Score,Best Params
0,Linear,0.654386,{'poly_features__degree': 1}
1,Lasso,0.652426,"{'lasso__alpha': 0.01, 'poly_features__degree': 2}"
2,Ridge,0.662915,"{'poly_features__degree': 1, 'ridge__alpha': 10}"


Looking at the model above, we can see that the models all achieved a similar R-squared value of around '0.65'. This means that my best models can explain about 65% of the variance in the dependent variables. Note that real world models will never achieve a perfect 100% score, as real world data tends to have many inaccuracies and complicating factors that cannot be easily quantified. This score is fairly good by this metric, and any of these three models could be considered as viable models for regression.

Looking at the performance of the models, the linear model performed well with no polynomial features, and seemed to struggle to converge with the higher polynomial features. In fact, only the Lasso model found success with the polynomial features greater than 1. The Lasso model only successfully converged when the alpha was low, which is when the performance is most similar to Linear Regression. Most of the models did not converge with higher polynomial features and with higher alpha values, so future research should focus on low alphas and ignore large polynomial features. 

The performance of the models is not significantly different enough to warrant any model being strictly better than the others. Given the nature of 5-fold cross validation, there is a degree of randomness in these scores, and in fact running the models again may produce slightly different results. Since the performance of the models is comparable, I would recommend that the next criteria for judgement be the time to fit and tune the model. Looking closely at the results, the Ridge regression model had the lowest mean fit time and mean score time. The score of the ridge model is slightly higher than the others, so combining these factors, the Ridge model is the one I would recommend for future analysis.